In [3]:
from _imports_ import *

## Load data

In [4]:
# The data, shuffled and split between train and test sets:
num_classes = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


## Simple CNN network

In [ ]:
batch_size = 32
epochs = 20
data_augmentation = True
filter_size = 4

model = Sequential()

model.add(Conv2D(32, (filter_size, filter_size), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (filter_size, filter_size)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (filter_size, filter_size), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (filter_size, filter_size)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
# opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)
opt = keras.optimizers.adam(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

## Residual CNN network

In [5]:
def getOutputShape(block):
    return block.output_shape[1:]

def getResBlock(input_shape, n_feature_maps, filter_size, stride_size=1):
    print("Build ResBlock: input_shape=%s n_features_maps=%d filter_size=%d stride_size=%d" % 
          (input_shape, n_feature_maps, filter_size, stride_size))
    x = Input(shape=input_shape)
    
    # ---- Shortcut path ----
    is_expand_channels = not (input_shape[0] == n_feature_maps)
    if stride_size > 1:
        print ('  - Input channels: %d ---> num feature maps on sub-sampled out: %d' % (input_shape[0], n_feature_maps)  )
        shortcut_y = Conv2D(n_feature_maps, kernel_size=1, strides=stride_size, padding='same')(x)
    else:
        if is_expand_channels:
            print ('  - Input channels: %d ---> num feature maps on out: %d' % (input_shape[0], n_feature_maps)  )
            shortcut_y = Conv2D(n_feature_maps, kernel_size=1, padding='same')(x)
        else:
            print ('  - Input channels: %d ---> num feature maps' % (n_feature_maps)  )
            shortcut_y = x
    
    # ---- Convolution path ----
    conv_y = x
    conv_y = Conv2D(n_feature_maps, kernel_size=filter_size, strides=stride_size, padding='same')(conv_y)
    conv_y = BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
    conv_y = Conv2D(n_feature_maps, kernel_size=filter_size, padding='same')(conv_y)
    conv_y = BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
#     conv_y = MaxPooling2D(pool_size=(2, 2))(conv_y)
    conv_y = Dropout(0.25)(conv_y)
    
    y = add([shortcut_y, conv_y])
    block = Model(inputs=x, outputs=y)
    return block

def getBigResBlock(input_shape, n1, n2, filter_size):
    block = Sequential()
    res1 = getResBlock(input_shape, n1, filter_size)
    block.add(res1)
    res2 = getResBlock(getOutputShape(res1), n2, filter_size, stride_size=1)
    block.add(res2)
#     block.add(AveragePooling2D(pool_size=2))
    block.add(MaxPooling2D(pool_size=2,strides=2))
    return block


batch_size = 16
epochs = 20
data_augmentation = True
filter_size = 3

data_input_shape = x_train.shape[1:]

model = Sequential()
model.add(Conv2D(128, kernel_size=filter_size, padding='same', input_shape=data_input_shape))
model.add(Activation('relu'))

res1 = getBigResBlock(getOutputShape(model), 128, 256, 2)
res2 = getBigResBlock(getOutputShape(res1), 256, 512, 2)
res3 = getBigResBlock(getOutputShape(res2), 512, 1024, 2)
model.add(res1)
model.add(res2)
model.add(res3)

model.add(BatchNormalization(axis=1))
model.add(Activation('relu'))
# [Classifier]
model.add(Flatten())
model.add(Dense(num_classes))
model.add(Activation('softmax'))

print(model.output_shape)
print(model.summary())

opt = keras.optimizers.adam(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Build ResBlock: input_shape=(32, 32, 128) n_features_maps=128 filter_size=2 stride_size=1
  - Input channels: 32 ---> num feature maps on out: 128
Build ResBlock: input_shape=(32, 32, 128) n_features_maps=256 filter_size=2 stride_size=1
  - Input channels: 32 ---> num feature maps on out: 256
Build ResBlock: input_shape=(16, 16, 256) n_features_maps=256 filter_size=2 stride_size=1
  - Input channels: 16 ---> num feature maps on out: 256
Build ResBlock: input_shape=(16, 16, 256) n_features_maps=512 filter_size=2 stride_size=1
  - Input channels: 16 ---> num feature maps on out: 512
Build ResBlock: input_shape=(8, 8, 512) n_features_maps=512 filter_size=2 stride_size=1
  - Input channels: 8 ---> num feature maps on out: 512
Build ResBlock: input_shape=(8, 8, 512) n_features_maps=1024 filter_size=2 stride_size=1
  - Input channels: 8 ---> num feature maps on out: 1024
(None, 10)
_________________________________________________________________
Layer (type)                 Output Shape    

In [6]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        validation_data=(x_test, y_test))

Using real-time data augmentation.
Epoch 1/20
3125/3125 [==============================] - 6610s - loss: 1.3468 - acc: 0.5206 - val_loss: 1.3596 - val_acc: 0.5293
Epoch 2/20
3125/3125 [==============================] - 6593s - loss: 0.8747 - acc: 0.6936 - val_loss: 0.9882 - val_acc: 0.6542
Epoch 3/20
3125/3125 [==============================] - 6591s - loss: 0.7009 - acc: 0.7571 - val_loss: 0.8483 - val_acc: 0.7131
Epoch 4/20
3024/3125 [============================>.] - ETA: 202s - loss: 0.5949 - acc: 0.7947

KeyboardInterrupt: 